In [ ]:
import numpy as np

from FeatureAcquisition import FeatureAcquisition
from Predictor import Predictor
from Generators import GaussianSampler


In [4]:
x = np.array([[2., 4., 3., 5., 6., 3., 3., 4.]])
m = np.array([[0., 0., 1., 1., 0., 0., 0., 0.]])
cost_per_feature = np.ones(8)
budget = 4

feature_dim = 8
num_classes = 3
hidden_dim = 8
predictor = Predictor(feature_dim, num_classes, hidden_dim)

generator = GaussianSampler(mean=0.0, std=1.0)

FA = FeatureAcquisition(
    x=x,
    m=m,
    generative_model=generator,
    num_samples=10,
    predictor=predictor,
    alpha=1.0,
    gamma=0.0
)

# 초기 mask 확인
print(FA.m)

# mask 하나씩 잘 고르는지 확인
for t in range(5):
    scores = FA.alpha_gamma_cmi()
    print(FA.x, FA.m, FA.acquire())

[[0. 0. 1. 1. 0. 0. 0. 0.]]
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[0. 0. 1. 1. 0. 1. 0. 0.]] (array([[0., 0., 1., 1., 0., 1., 0., 0.]]), array([5]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[0. 0. 1. 1. 0. 1. 1. 0.]] (array([[0., 0., 1., 1., 0., 1., 1., 0.]]), array([6]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 0. 1. 1. 0. 1. 1. 0.]] (array([[1., 0., 1., 1., 0., 1., 1., 0.]]), array([0]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 1. 1. 1. 0. 1. 1. 0.]] (array([[1., 1., 1., 1., 0., 1., 1., 0.]]), array([1]))
[[2. 4. 3. 5. 6. 3. 3. 4.]] [[1. 1. 1. 1. 1. 1. 1. 0.]] (array([[1., 1., 1., 1., 1., 1., 1., 0.]]), array([4]))


In [2]:
import torch
import os
from pathlib import Path
import numpy as np

# 데이터 폴더 경로
data_dir = Path('./data')

# 모든 데이터셋 폴더 찾기
datasets = [d for d in data_dir.iterdir() if d.is_dir()]

print(f"총 {len(datasets)}개의 데이터셋을 찾았습니다.\n")
print("="*80)

# 각 데이터셋별로 dataset_dict 확인
for dataset_path in sorted(datasets):
    dataset_name = dataset_path.name
    dict_path = dataset_path / 'dataset_dict.pt'
    
    print(f"\n 데이터셋: {dataset_name}")
    print("-"*80)
    
    if dict_path.exists():
        try:
            # dataset_dict 로드
            dataset_dict = torch.load(dict_path)
            
            print(f"✓ dataset_dict.pt 존재")
            print(f"\n📊 내용:")
            
            # dict인 경우
            if isinstance(dataset_dict, dict):
                print(f"  타입: dict")
                print(f"  키(keys): {list(dataset_dict.keys())}")
                print(f"\n  각 키별 정보:")
                for key, value in dataset_dict.items():
                    if isinstance(value, torch.Tensor):
                        print(f"    - {key}: shape={value.shape}, dtype={value.dtype}")
                    elif isinstance(value, np.ndarray):
                        print(f"    - {key}: shape={value.shape}, dtype={value.dtype}")
                    elif isinstance(value, list):
                        print(f"    - {key}: list, length={len(value)}")
                    else:
                        print(f"    - {key}: type={type(value).__name__}, value={value}")
            else:
                print(f"  타입: {type(dataset_dict).__name__}")
                print(f"  값: {dataset_dict}")
                
        except Exception as e:
            print(f"✗ 로드 중 에러 발생: {str(e)}")
    else:
        print(f"✗ dataset_dict.pt 파일이 존재하지 않습니다.")
        # 폴더 내 파일 목록 출력
        files = list(dataset_path.glob('*'))
        if files:
            print(f"  폴더 내 파일: {[f.name for f in files]}")
    
    print("="*80)


총 7개의 데이터셋을 찾았습니다.


 데이터셋: FashionMNIST
--------------------------------------------------------------------------------
✓ dataset_dict.pt 존재

📊 내용:
  타입: dict
  키(keys): ['dataset', 'num_con_features', 'num_cat_features', 'most_categories', 'out_dim', 'metric', 'max_dim', 'log_class_probs']

  각 키별 정보:
    - dataset: type=str, value=fashion_mnist
    - num_con_features: type=int, value=20
    - num_cat_features: type=int, value=0
    - most_categories: type=int, value=0
    - out_dim: type=int, value=10
    - metric: type=str, value=accuracy
    - max_dim: type=NoneType, value=None
    - log_class_probs: shape=torch.Size([10]), dtype=torch.float32

 데이터셋: bank_marketing
--------------------------------------------------------------------------------
✓ dataset_dict.pt 존재

📊 내용:
  타입: dict
  키(keys): ['dataset', 'num_con_features', 'num_cat_features', 'most_categories', 'out_dim', 'metric', 'max_dim', 'log_class_probs']

  각 키별 정보:
    - dataset: type=str, value=bank
    - num_con_feat

In [ ]:
import numpy as np

# 예제: m_np 행렬에서 랜덤하게 k개 요소를 0으로 바꾸기

# 테스트용 행렬 생성
m_np = np.array([[1., 2., 3., 4., 5.],
                 [6., 7., 8., 9., 10.]])

print("원본 행렬:")
print(m_np)
print(f"shape: {m_np.shape}")

# 0으로 바꿀 요소 개수
k = 3

# 방법 1: flatten된 인덱스를 사용 (가장 간단)
m_np_copy1 = m_np.copy()
flat_indices = np.random.choice(m_np.size, size=k, replace=False)
m_np_copy1.flat[flat_indices] = 0

print(f"\n방법 1: 랜덤하게 {k}개 요소를 0으로 변경")
print(m_np_copy1)
print(f"선택된 flat indices: {flat_indices}")

# 방법 2: 2D 인덱스를 직접 사용
m_np_copy2 = m_np.copy()
rows, cols = m_np.shape
total_elements = rows * cols
flat_indices = np.random.choice(total_elements, size=k, replace=False)
row_indices = flat_indices // cols
col_indices = flat_indices % cols
m_np_copy2[row_indices, col_indices] = 0

print(f"\n방법 2: 2D 인덱스 사용")
print(m_np_copy2)
print(f"선택된 (row, col) 인덱스: {list(zip(row_indices, col_indices))}")

# 방법 3: 0이 아닌 요소 중에서만 선택하기
m_np_copy3 = m_np.copy()
non_zero_mask = m_np_copy3 != 0
non_zero_indices = np.where(non_zero_mask)
num_non_zero = len(non_zero_indices[0])

if num_non_zero >= k:
    selected = np.random.choice(num_non_zero, size=k, replace=False)
    selected_rows = non_zero_indices[0][selected]
    selected_cols = non_zero_indices[1][selected]
    m_np_copy3[selected_rows, selected_cols] = 0
    
    print(f"\n방법 3: 0이 아닌 요소 중에서만 선택")
    print(m_np_copy3)
    print(f"선택된 (row, col) 인덱스: {list(zip(selected_rows, selected_cols))}")
else:
    print(f"\n방법 3: 0이 아닌 요소가 {num_non_zero}개뿐이라 {k}개를 선택할 수 없습니다.")

# 실용적인 함수로 만들기
def set_random_elements_to_zero(matrix, k, only_nonzero=False):
    """
    행렬에서 랜덤하게 k개 요소를 0으로 설정
    
    Parameters:
    -----------
    matrix : np.ndarray
        입력 행렬
    k : int
        0으로 바꿀 요소 개수
    only_nonzero : bool
        True면 0이 아닌 요소 중에서만 선택
    
    Returns:
    --------
    np.ndarray : 수정된 행렬
    tuple : 선택된 (row_indices, col_indices)
    """
    result = matrix.copy()
    
    if only_nonzero:
        non_zero_indices = np.where(result != 0)
        num_available = len(non_zero_indices[0])
        if num_available < k:
            raise ValueError(f"0이 아닌 요소가 {num_available}개뿐이라 {k}개를 선택할 수 없습니다.")
        selected = np.random.choice(num_available, size=k, replace=False)
        row_indices = non_zero_indices[0][selected]
        col_indices = non_zero_indices[1][selected]
    else:
        flat_indices = np.random.choice(result.size, size=k, replace=False)
        row_indices = flat_indices // result.shape[1]
        col_indices = flat_indices % result.shape[1]
    
    result[row_indices, col_indices] = 0
    return result, (row_indices, col_indices)

# 함수 테스트
print("\n" + "="*60)
print("함수 버전 테스트:")
result, (rows, cols) = set_random_elements_to_zero(m_np, k=4, only_nonzero=True)
print(f"\n랜덤하게 4개 요소를 0으로 변경:")
print(result)
print(f"선택된 위치: {list(zip(rows, cols))}")
